In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
def generate_key_for_evaluation(df):
    left_table_name = []
    left_column_name = []
    right_table_name = []
    right_column_name = []
    df_key = pd.DataFrame((), columns=df.columns)
    
    for i in range(df.shape[0]):
        rtn = df.iloc[i, 2]
        ltn = df.iloc[i, 0]
        right_table_name.append(rtn if rtn > ltn else ltn)
        left_table_name.append(rtn if rtn < ltn else ltn)
    
        rcn = df.iloc[i, 3]
        lcn = df.iloc[i, 1]
        right_column_name.append(rcn if rtn > ltn else lcn)
        left_column_name.append(rcn if rtn < ltn else lcn)
    
    d = {'LEFT_TABLE':left_table_name,
         'LEFT_COLUMN':left_column_name,
         'RIGHT_TABLE':right_table_name,
         'RIGHT_COLUMN':right_column_name}
    df_key = pd.DataFrame(d)
    df_key = df_key.drop_duplicates()
    df_key = df_key.reset_index(drop=True)
    
    key = []
    for i in range(df_key.shape[0]):
        key.append("#".join(df_key.iloc[i,:]))
    df_key['KEY'] = key

    return df_key

In [5]:
def fix_table_names(table_name):
    return table_name + '.csv'

In [6]:
cta_matches = pd.read_csv('Description_test/CTA_from_descriptions/junio_description_cta_matches.csv')
cta_matches = cta_matches[['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN']]
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN
0,datasets_579296_1047868_authors,Author_ID,eo_xx,EIN
1,datasets_517172_952401_train,id_code,eo_xx,EIN
2,eo_pr,EIN,eo_xx,EIN
3,eo_pr,GROUP,eo_xx,EIN
4,eo_pr,SUBSECTION,eo_xx,EIN


In [7]:
cta_matches['LEFT_TABLE'] = fix_table_names(cta_matches['LEFT_TABLE'])
cta_matches['RIGHT_TABLE'] = fix_table_names(cta_matches['RIGHT_TABLE'])

In [8]:
cta_matches = generate_key_for_evaluation(cta_matches)
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,datasets_579296_1047868_authors.csv,Author_ID,eo_xx.csv,EIN,datasets_579296_1047868_authors.csv#Author_ID#...
1,datasets_517172_952401_train.csv,id_code,eo_xx.csv,EIN,datasets_517172_952401_train.csv#id_code#eo_xx...
2,eo_pr.csv,EIN,eo_xx.csv,EIN,eo_pr.csv#EIN#eo_xx.csv#EIN
3,eo_pr.csv,GROUP,eo_xx.csv,EIN,eo_pr.csv#GROUP#eo_xx.csv#EIN
4,eo_pr.csv,SUBSECTION,eo_xx.csv,EIN,eo_pr.csv#SUBSECTION#eo_xx.csv#EIN


In [9]:
cta_matches.shape

(2841, 5)

In [10]:
groundTruth = pd.read_csv('joinable_columns_gt3_quality.csv')
groundTruth = groundTruth[['ds_name', 'att_name', 'ds_name_2', 'att_name_2']]
groundTruth = generate_key_for_evaluation(groundTruth)
groundTruth.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,road-ahead-current-road-closures.csv,COMP_DATE,road-ahead-upcoming-projects.csv,COMP_DATE,road-ahead-current-road-closures.csv#COMP_DATE...
1,eo_pr.csv,NTEE_CD,eo_xx.csv,NTEE_CD,eo_pr.csv#NTEE_CD#eo_xx.csv#NTEE_CD
2,eo4.csv,NAME,eo_pr.csv,NAME,eo4.csv#NAME#eo_pr.csv#NAME
3,eo4.csv,ICO,eo_pr.csv,ICO,eo4.csv#ICO#eo_pr.csv#ICO
4,eo4.csv,STREET,eo_pr.csv,STREET,eo4.csv#STREET#eo_pr.csv#STREET


In [11]:
groundTruth.shape

(53, 5)

In [12]:
tp = 0
fp = 0
fn = 0

for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] in groundTruth.iloc[:,4].values):
        tp += 1
    else:
        fp += 1
    
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        fn += 1
    
precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2 * (precision*recall)/(precision+recall)

print(precision, recall, f1_score)

0.017247448081661388 0.9245283018867925 0.03386316516931583


In [13]:
tp, fp, fn

(49, 2792, 4)

# Falso Positivo

In [14]:
fp_rows = []
for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] not in groundTruth.iloc[:,4].values):
        fp_rows.append(cta_matches.iloc[i,4])

fp_df = pd.DataFrame(list(map(lambda x: x.split('#'), fp_rows)), columns=['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN'])
# fp_df.to_csv('false_positives.csv', index=False)

# Falso Negativo

In [15]:
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        print(groundTruth.iloc[i,4])

eo4.csv#ICO#eo_pr.csv#ICO
eo4.csv#CITY#eo_pr.csv#CITY
eo4.csv#ICO#eo_xx.csv#ICO
eo4.csv#SORT_NAME#eo_xx.csv#SORT_NAME
